In [2]:
!pip install geopy
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from scipy.spatial.distance import pdist
from scipy.spatial import cKDTree
from sklearn.neighbors import BallTree
from pyproj import Geod 
from geopy.distance import geodesic

from collections import Counter
from tqdm import tqdm
from shapely import wkt
from shapely import wkb

geod = Geod(ellps="WGS84")

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 6.3 MB/s eta 0:00:00


In [3]:
# Constants
EARTH_RADIUS = 6371  # Earth radius in km

In [4]:
df_K = pd.read_parquet("Churu_height_NEW_GHSL_SMOD_elevations_BDensity.parquet")
df_K["geometry"] = df_K["geometry"].apply(wkb.loads)
df_K = gpd.GeoDataFrame(df_K, geometry='geometry')


#df_K = gpd.GeoDataFrame(df_K, geometry=shapely.from_wkb(df_K['geometry']))
df_K.index = [i for i in range(len(df_K))]



### Perimiter to area ratio

In [5]:
df_K.head()


,perimeter_in_meters,building_faces,bf_source,confidence,geometry,longitude,latitude,id,area_in_meters,height_mean,...,height,floors,gfa_in_meters,urban_split,ghsl_smod,elevation,building_density_50,building_density_100,building_density_250,building_density_500
0,10.003668,4,google,0.7370,"POLYGON ((74.46928 28.43321, 74.46928 28.43323...",74.469271,28.433219,74.46927144167354:28.433219462727678,6.254586,0.000000,...,4.5,1.0,6.25459,Urban,Urban Center,255.0,0,0,87,583
1,14.526151,4,google,0.7366,"POLYGON ((74.46927 28.43434, 74.46927 28.43437...",74.469250,28.434354,74.46924985073832:28.434353850621235,13.133803,1.833333,...,4.5,1.0,13.13380,Urban,Urban Center,253.0,3,6,133,720
2,28.127545,4,google,0.7958,"POLYGON ((74.4668 28.43541, 74.46678 28.43548,...",74.466757,28.435439,74.4667571690444:28.43543941637925,48.951191,1.554054,...,4.5,1.0,48.95119,Urban,Urban Center,253.0,4,4,5,158
3,25.103799,4,google,0.8284,"POLYGON ((74.46704 28.43538, 74.46703 28.43542...",74.466999,28.435397,74.46699939219799:28.435397468510892,36.518629,4.586735,...,4.5,1.0,36.51863,Urban,Urban Center,254.0,2,4,5,191
4,26.080669,4,google,0.7482,"POLYGON ((74.46708 28.43545, 74.46708 28.43549...",74.467039,28.435467,74.46703907487851:28.43546728639071,39.264648,2.066667,...,4.5,1.0,39.26465,Urban,Urban Center,253.0,2,4,5,202


In [6]:

df_K['building_perimeter_in_meters_new'] = df_K["geometry"].apply(lambda g: geod.geometry_area_perimeter(g)[1]) #this extracts second value from the tuple which is perimeter

In [7]:
df_K['perimeter_to_area_ratio'] = df_K['building_perimeter_in_meters_new'] / df_K['area_in_meters']
df_K

,perimeter_in_meters,building_faces,bf_source,confidence,geometry,longitude,latitude,id,area_in_meters,height_mean,...,gfa_in_meters,urban_split,ghsl_smod,elevation,building_density_50,building_density_100,building_density_250,building_density_500,building_perimeter_in_meters_new,perimeter_to_area_ratio
0,10.003668,4,google,0.7370,"POLYGON ((74.46928 28.43321, 74.46928 28.43323...",74.469271,28.433219,74.46927144167354:28.433219462727678,6.254586,0.000000,...,6.25459,Urban,Urban Center,255.0,0,0,87,583,10.003668,1.599413
1,14.526151,4,google,0.7366,"POLYGON ((74.46927 28.43434, 74.46927 28.43437...",74.469250,28.434354,74.46924985073832:28.434353850621235,13.133803,1.833333,...,13.13380,Urban,Urban Center,253.0,3,6,133,720,14.526151,1.106013
2,28.127545,4,google,0.7958,"POLYGON ((74.4668 28.43541, 74.46678 28.43548,...",74.466757,28.435439,74.4667571690444:28.43543941637925,48.951191,1.554054,...,48.95119,Urban,Urban Center,253.0,4,4,5,158,28.127545,0.574604
3,25.103799,4,google,0.8284,"POLYGON ((74.46704 28.43538, 74.46703 28.43542...",74.466999,28.435397,74.46699939219799:28.435397468510892,36.518629,4.586735,...,36.51863,Urban,Urban Center,254.0,2,4,5,191,25.103799,0.687424
4,26.080669,4,google,0.7482,"POLYGON ((74.46708 28.43545, 74.46708 28.43549...",74.467039,28.435467,74.46703907487851:28.43546728639071,39.264648,2.066667,...,39.26465,Urban,Urban Center,253.0,2,4,5,202,26.080669,0.664228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481129,15.665885,4,microsoft,NaN,"POLYGON ((75.61065 28.47085, 75.61061 28.47085...",75.610628,28.470834,75.61062820689784:28.470834024537673,15.265261,0.000000,...,15.26526,Rural,Low Density Rural,273.0,3,10,23,31,15.665885,1.026244
1481130,24.967117,4,microsoft,NaN,"POLYGON ((75.61069 28.47235, 75.61071 28.47243...",75.610679,28.472398,75.610678770702:28.47239797637667,31.108396,1.560811,...,31.10840,Rural,Low Density Rural,273.0,4,4,10,34,24.967117,0.802585
1481131,19.878225,4,microsoft,NaN,"POLYGON ((75.6127 28.48045, 75.6127 28.48048, ...",75.612672,28.480463,75.6126724498209:28.48046310058587,24.061042,0.000000,...,24.06104,Rural,Low Density Rural,271.0,3,3,3,5,19.878225,0.826158
1481132,18.284218,4,microsoft,NaN,"POLYGON ((75.61294 28.48055, 75.61288 28.48055...",75.612911,28.480535,75.61291105158907:28.48053476788568,20.038088,0.000000,...,20.03809,Rural,Low Density Rural,271.0,3,3,3,5,18.284218,0.912473


In [8]:
df_K['perimeter_to_area_ratio'] = df_K['perimeter_to_area_ratio'].clip(upper=6.5)
print(df_K['perimeter_to_area_ratio'].describe())

count    1.481134e+06
mean     8.178839e-01
std      2.843311e-01
min      4.667845e-02
25%      6.139691e-01
50%      8.113813e-01
75%      1.000433e+00
max      4.935747e+00
Name: perimeter_to_area_ratio, dtype: float64


In [9]:
df_K['perimeter_to_area_ratio'].max()

np.float64(4.935746739100953)

In [10]:
df_K['normalized_perimeter_to_area_ratio'] = df_K['perimeter_to_area_ratio'] / df_K['perimeter_to_area_ratio'].max()

### Radius calculation

df_K['geo_original']=df_K['geometry']

In [11]:
df_K.set_crs("EPSG:4326", inplace=True, allow_override=True)  
print("Current CRS:", df_K.crs)

df_K = df_K.to_crs("EPSG:7767") 

df_K["centroid"] = df_K.geometry.centroid

print(df_K[["geometry", "centroid"]].head())


Current CRS: EPSG:4326
                                            geometry  \
0  POLYGON ((773560.12 2063675.965, 773560.142 20...   
1  POLYGON ((773560.204 2063802.831, 773559.956 2...   
2  POLYGON ((773316.34 2063926.528, 773314.891 20...   
3  POLYGON ((773340.849 2063923.233, 773339.99 20...   
4  POLYGON ((773344.996 2063930.767, 773344.342 2...   

                         centroid  
0  POINT (773558.864 2063677.243)  
1  POINT (773558.361 2063804.676)  
2  POINT (773312.464 2063929.789)  
3  POINT (773336.451 2063924.767)  
4  POINT (773340.492 2063932.558)  


In [12]:

def calculate_radius(geometry):
    
    geometry = geometry if geometry.type == 'Polygon' else geometry.convex_hull
    centroid = geometry.centroid
    boundary_points = np.array(geometry.exterior.coords)
    distances = np.linalg.norm(boundary_points - np.array([centroid.x, centroid.y]), axis=1)
    return np.mean(distances)

# Compute radius in meters
df_K["radius_m"] = df_K["geometry"].apply(calculate_radius)

# Convert back to WGS84 (if needed)
df_K = df_K.to_crs("EPSG:4326")

print(df_K[["centroid", "radius_m"]].head())


/tmp/ipykernel_135128/2677447443.py:3: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  geometry = geometry if geometry.type == 'Polygon' else geometry.convex_hull


                         centroid  radius_m
0  POINT (773558.864 2063677.243)  1.792272
1  POINT (773558.361 2063804.676)  2.607885
2  POINT (773312.464 2063929.789)  5.064631
3  POINT (773336.451 2063924.767)  4.658587
4  POINT (773340.492 2063932.558)  4.847894


In [13]:
df_K["num_vertices"] = df_K["geometry"].apply(lambda x: len(x.exterior.coords) if x.type == 'Polygon' else sum(len(g.exterior.coords) for g in x.geoms))
# df_K["num_vertices"] = df_K["geometry"].apply(lambda x: len(x.exterior.coords))
df_K[["num_vertices"]].describe()

/tmp/ipykernel_135128/2329847680.py:1: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  df_K["num_vertices"] = df_K["geometry"].apply(lambda x: len(x.exterior.coords) if x.type == 'Polygon' else sum(len(g.exterior.coords) for g in x.geoms))


,num_vertices
count,1.481134e+06
mean,5.375669e+00
std,1.040535e+00
min,4.000000e+00
25%,5.000000e+00
50%,5.000000e+00
75%,5.000000e+00
max,4.200000e+01


### Roads calculation

In [14]:
#file from extract_roads
final_gdf = gpd.read_parquet('roads_inside_Churu.parquet')
final_gdf = final_gdf.set_crs("EPSG:4326")
final_gdf['road_index'] = [i for i in range(len(final_gdf))]

CHECKS

In [15]:
final_gdf

geometry  \
element id                                                              
node    359006996                            POINT (74.46892 27.7118)   
        359007349                            POINT (74.9707 28.29253)   
        359205518                           POINT (74.51479 28.44484)   
        1398553812                          POINT (75.36486 28.62537)   
        1398553859                          POINT (75.36701 28.62631)   
...                                                               ...   
way     1433970014  LINESTRING (75.42167 28.90515, 75.42169 28.904...   
        1433970015  LINESTRING (75.4217 28.90478, 75.42201 28.9047...   
        1433970016  LINESTRING (75.42179 28.90384, 75.42153 28.903...   
        1433970017  LINESTRING (75.41133 28.9002, 75.41153 28.9002...   
        1434287210  LINESTRING (75.50904 28.91637, 75.50901 28.916...   

                            highway        name  direction  historic  \
element id                                                             
node    359006996   mini_roundabout        None  clockwise      None   
        359007349   mini_roundabout  Dharmstoop  clockwise  monument   
        359205518   mini_roundabout        None       None      None   
        1398553812   turning_circle        None       None      None   
        1398553859   turning_circle        None       None      None   
...                             ...         ...        ...       ...   
way     1433970014      residential        None       None      None   
        1433970015      residential        None       None      None   
        1433970016      residential        None       None      None   
        1433970017      residential        None       None      None   
        1434287210     unclassified        None       None      None   

                       name:en     name:hi   ref ref:old lanes  ... cycleway  \
element id                                                      ...            
node    359006996         None        None  None    None  None  ...     None   
        359007349   Dharmstoop  धर्म स्तूप  None    None  None  ...     None   
        359205518         None        None  None    None  None  ...     None   
        1398553812        None        None  None    None  None  ...     None   
        1398553859        None        None  None    None  None  ...     None   
...                        ...         ...   ...     ...   ...  ...      ...   
way     1433970014        None        None  None    None  None  ...     None   
        1433970015        None        None  None    None  None  ...     None   
        1433970016        None        None  None    None  None  ...     None   
        1433970017        None        None  None    None  None  ...     None   
        1434287210        None        None  None    None  None  ...     None   

                   tracktype is_in:state network  note route  type wikidata  \
element id                                                                    
node    359006996       None        None    None  None  None  None     None   
        359007349       None        None    None  None  None  None     None   
        359205518       None        None    None  None  None  None     None   
        1398553812      None        None    None  None  None  None     None   
        1398553859      None        None    None  None  None  None     None   
...                      ...         ...     ...   ...   ...   ...      ...   
way     1433970014      None        None    None  None  None  None     None   
        1433970015      None        None    None  None  None  None     None   
        1433970016      None        None    None  None  None  None     None   
        1433970017      None        None    None  None  None  None     None   
        1434287210      None        None    None  None  None  None     None   

                   wikipedia road_index  
element id                               
node    359006996    

In [16]:
len(list(final_gdf.columns))

43

In [17]:
print(final_gdf.geometry.type.value_counts())

print("Empty geometries count:", final_gdf.geometry.is_empty.sum())

print("Unique highway values:", final_gdf['highway'].unique())

LineString    7367
Point           12
Name: count, dtype: int64
Empty geometries count: 0
Unique highway values: ['mini_roundabout' 'turning_circle' 'bus_stop' 'primary' 'trunk'
 'tertiary' 'residential' 'unclassified' 'secondary' 'track' 'trunk_link'
 'service' 'living_street' 'path' 'steps' 'primary_link' 'footway'
 'cycleway' 'pedestrian']


CALCULATIONS

In [18]:
#extraction of required columns
required_columns = {"highway", "geometry", "id",'width','oneway','junction','lanes','maxspeed','motorcar', 'road_index'}
final_gdf = final_gdf[[col for col in required_columns if col in final_gdf.columns]]

final_gdf

road_index  \
element id                       
node    359006996            0   
        359007349            1   
        359205518            2   
        1398553812           3   
        1398553859           4   
...                        ...   
way     1433970014        7374   
        1433970015        7375   
        1433970016        7376   
        1433970017        7377   
        1434287210        7378   

                                                             geometry lanes  \
element id                                                                    
node    359006996                            POINT (74.46892 27.7118)  None   
        359007349                            POINT (74.9707 28.29253)  None   
        359205518                           POINT (74.51479 28.44484)  None   
        1398553812                          POINT (75.36486 28.62537)  None   
        1398553859                          POINT (75.36701 28.62631)  None   
...                                                               ...   ...   
way     1433970014  LINESTRING (75.42167 28.90515, 75.42169 28.904...  None   
        1433970015  LINESTRING (75.4217 28.90478, 75.42201 28.9047...  None   
        1433970016  LINESTRING (75.42179 28.90384, 75.42153 28.903...  None   
        1433970017  LINESTRING (75.41133 28.9002, 75.41153 28.9002...  None   
        1434287210  LINESTRING (75.50904 28.91637, 75.50901 28.916...  None   

                            highway oneway junction maxspeed  
element id                                                    
node    359006996   mini_roundabout   None     None     None  
        359007349   mini_roundabout   None     None     None  
        359205518   mini_roundabout   None     None     None  
        1398553812   turning_circle   None     None     None  
        1398553859   turning_circle   None     None     None  
...                             ...    ...      ...      ...  
way     1433970014      residential   None     None     None  
        1433970015      residential   None     None     None  
        1433970016      residential   None     None     None  
        1433970017      residential   None     None     None  
        1434287210     unclassified   None     None     None  

[7379 rows x 7 columns]

In [19]:
duplic=final_gdf[final_gdf.duplicated(keep=False)]
print(duplic.shape)

(0, 7)


In [20]:
tolerance = 0.00001

final_gdf['geometry_simplified'] = final_gdf['geometry'].simplify(tolerance)
final_gdf

/usr/local/lib/python3.12/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


road_index  \
element id                       
node    359006996            0   
        359007349            1   
        359205518            2   
        1398553812           3   
        1398553859           4   
...                        ...   
way     1433970014        7374   
        1433970015        7375   
        1433970016        7376   
        1433970017        7377   
        1434287210        7378   

                                                             geometry lanes  \
element id                                                                    
node    359006996                            POINT (74.46892 27.7118)  None   
        359007349                            POINT (74.9707 28.29253)  None   
        359205518                           POINT (74.51479 28.44484)  None   
        1398553812                          POINT (75.36486 28.62537)  None   
        1398553859                          POINT (75.36701 28.62631)  None   
...                                                               ...   ...   
way     1433970014  LINESTRING (75.42167 28.90515, 75.42169 28.904...  None   
        1433970015  LINESTRING (75.4217 28.90478, 75.42201 28.9047...  None   
        1433970016  LINESTRING (75.42179 28.90384, 75.42153 28.903...  None   
        1433970017  LINESTRING (75.41133 28.9002, 75.41153 28.9002...  None   
        1434287210  LINESTRING (75.50904 28.91637, 75.50901 28.916...  None   

                            highway oneway junction maxspeed  \
element id                                                     
node    359006996   mini_roundabout   None     None     None   
        359007349   mini_roundabout   None     None     None   
        359205518   mini_roundabout   None     None     None   
        1398553812   turning_circle   None     None     None   
        1398553859   turning_circle   None     None     None   
...                             ...    ...      ...      ...   
way     1433970014      residential   None     None     None   
        1433970015      residential   None     None     None   
        1433970016      residential   None     None     None   
        1433970017      residential   None     None     None   
        1434287210     unclassified   None     None     None   

                                                  geometry_simplified  
element id                                                             
node    359006996                            POINT (74.46892 27.7118)  
        359007349                            POINT (74.9707 28.29253)  
        359205518                           POINT (74.51479 28.44484)  
        1398553812                          POINT (75.36486 28.62537)  
        1398553859                          POINT (75.36701 28.62631)  
...                                                               ...  
way     1433970014  LINESTRING (75.42167 28.90515, 75.42179 28.903...  
        1433970015  LINESTRING (75.4217 28.90478, 75.42221 28.9048...  
        1433970016  LINESTRING (75.42179 28.90384, 75.42048 28.903...  
        1433970017  LINESTRING (75.41133 28.9002, 75.41167 28.9002...  
        1434287210  LINESTRING (75.50904 28.91637, 75.50855 28.914...  

[7379 rows x 8 columns]

In [21]:
roads_categories = {
    1: ['motorway', 'trunk_link', 'motorway_link', 'trunk', 'primary', 'primary_link'],
    2: ['secondary', 'secondary_link',],
    3: ['tertiary', 'tertiary_link', ],
    4: ['residential', 'footway', 'service', 'unclassified','living_street','steps','path','track','pedestrian','cycleway','raceway','bridleway','construction','services','bus_stop','road','rest_area','yes','emergency_access_point','corridor','junction','proposed','minor']
    }

In [22]:
def explode_multilinestrings(gdf):
    """ Convert MultiLineStrings to separate LineStrings """
    gdf = gdf.explode(ignore_index=True)
    return gdf[gdf.geometry.type == 'LineString'] 

final_gdf = final_gdf[final_gdf.geometry.notnull()]
final_gdf = explode_multilinestrings(final_gdf)
final_gdf

,road_index,lanes,highway,oneway,junction,maxspeed,geometry_simplified,geometry
12,12,None,primary,None,None,None,"LINESTRING (74.43911 27.81268, 74.43889 27.812...","LINESTRING (74.43911 27.81268, 74.439 27.81268..."
13,13,None,primary,None,None,None,"LINESTRING (74.43911 27.81268, 74.43889 27.812...","LINESTRING (74.43911 27.81268, 74.439 27.81268..."
14,14,2,trunk,None,None,None,"LINESTRING (74.95079 28.01506, 74.95082 28.015...","LINESTRING (74.95079 28.01506, 74.95082 28.015..."
15,15,2,primary,None,None,None,"LINESTRING (74.51479 28.44484, 74.5144 28.4453...","LINESTRING (74.51479 28.44484, 74.5144 28.4453..."
16,16,None,primary,None,None,None,"LINESTRING (74.51855 28.43882, 74.51866 28.438...","LINESTRING (74.51855 28.43882, 74.51866 28.438..."
...,...,...,...,...,...,...,...,...
7374,7374,None,residential,None,None,None,"LINESTRING (75.42167 28.90515, 75.42179 28.903...","LINESTRING (75.42167 28.90515, 75.42169 28.904..."
7375,7375,None,residential,None,None,None,"LINESTRING (75.4217 28.90478, 75.42221 28.9048...","LINESTRING (75.4217 28.90478, 75.42201 28.9047..."
7376,7376,None,residential,None,None,None,"LINESTRING (75.42179 28.90384, 75.42048 28.903...","LINESTRING (75.42179 28.90384, 75.42153 28.903..."
7377,7377,None,residential,None,None,None,"LINESTRING (75.41133 28.9002, 75.41167 28.9002...","LINESTRING (75.41133 28.9002, 75.41153 28.9002..."


In [23]:
projected_crs = "EPSG:3857"
final_gdf = final_gdf.to_crs(projected_crs)
df_K = df_K.to_crs("EPSG:3857")


In [24]:
df_K['centroid_x'] = df_K.geometry.apply(lambda g: g.centroid.xy[0][0])
df_K['centroid_y'] = df_K.geometry.apply(lambda g: g.centroid.xy[1][0])


In [25]:
def geodesic_distance(house_row, road_row):
    house_coords = (house_row['latitude'], house_row['longitude'])  
    road_coords = (road_row.geometry_centroid.y, road_row.geometry_centroid.x)  
    return geodesic(house_coords, road_coords).meters  # Returns distance in meters

In [26]:
def explode_road_geometry(df):
    
    road_rows = []
    for row_idx, row in df.to_dict(orient='index').items():
        
        for x, y in row['geometry'].coords:
            current_row = row.copy()
            current_row['geometry_centroid'] = shapely.Point(x, y)
            current_row['row_idx'] = row_idx
            road_rows.append(current_row)

    result_df = pd.DataFrame.from_dict(road_rows)
    result_df.index = [i for i in range(len(result_df))]    
    
    result_df['centroid_x'] = result_df.geometry.apply(lambda g: g.centroid.xy[0][0])
    result_df['centroid_y'] = result_df.geometry.apply(lambda g: g.centroid.xy[1][0]) 
    
    return result_df

In [27]:
category_bbox_size = {
    1: 5_000,
    2: 4_000, 
    3: 3_000,
    4: 2_000
}

In [28]:
df_K = df_K.sort_values(by='area_in_meters', ascending=True)
df_K.index = [i for i in range(len(df_K))]
df_K

,perimeter_in_meters,building_faces,bf_source,confidence,geometry,longitude,latitude,id,area_in_meters,height_mean,...,building_density_250,building_density_500,building_perimeter_in_meters_new,perimeter_to_area_ratio,normalized_perimeter_to_area_ratio,centroid,radius_m,num_vertices,centroid_x,centroid_y
0,7.243843,4,google,0.7724,"POLYGON ((8263508.924 3214523.343, 8263508.858...",74.232350,27.726409,74.23234975129046:27.72640873664523,2.504874,0.000000,...,689,1299,7.243843,2.891899,0.585909,POINT (748910.13 1984700.235),1.440035,5,8.263507e+06,3.214524e+06
1,9.011801,4,google,0.7108,"POLYGON ((8331698.419 3359491.171, 8331698.076...",74.844915,28.873013,74.84491545220574:28.873013019801967,2.513892,1.000000,...,551,2090,9.011801,3.584800,0.726293,POINT (811384.793 2112666.72),1.983237,5,8.331698e+06,3.359493e+06
2,6.354292,4,google,0.7123,"POLYGON ((8262863.432 3202487.532, 8262863.149...",74.226555,27.630664,74.22655544425939:27.630663527650903,2.514733,1.722222,...,20,42,6.354292,2.526826,0.511944,POINT (748179.381 1973979.881),1.137805,5,8.262862e+06,3.202488e+06
3,6.957619,4,google,0.7604,"POLYGON ((8350523.934 3291257.556, 8350523.917...",75.014020,28.334861,75.0140202674914:28.334861398658834,2.529426,0.000000,...,1,22,6.957619,2.750671,0.557296,POINT (827533.711 2052018.279),1.344463,5,8.350523e+06,3.291258e+06
4,8.217702,4,google,0.7845,"POLYGON ((8301105.491 3221600.715, 8301105.038...",74.570094,27.782683,74.57009354210085:27.782682537523247,2.530687,2.500000,...,506,1338,8.217702,3.247222,0.657899,POINT (782669.758 1990560.645),1.739007,5,8.301105e+06,3.221603e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481129,288.063775,4,google,0.9623,"POLYGON ((8290221.138 3301951.19, 8290174.466 ...",74.471835,28.419541,74.47183471063026:28.41954127037166,5116.472895,10.031507,...,9,48,288.063775,0.056301,0.011407,POINT (773793.576 2062137.77),51.954061,5,8.290167e+06,3.301972e+06
1481130,309.362995,6,google,0.8996,"POLYGON ((8344528.413 3273598.037, 8344531.568...",74.960137,28.195540,74.96013678708111:28.195540360926653,5519.215490,6.671840,...,48,58,309.362995,0.056052,0.011356,POINT (822022.567 2036431.841),52.343864,7,8.344524e+06,3.273649e+06
1481131,444.405058,21,google,0.9111,"POLYGON ((8391235.056 3328671.415, 8391232.696...",75.380226,28.630323,75.38022567723831:28.630323167273463,5533.380311,8.208935,...,188,784,444.405058,0.080313,0.016272,POINT (864171.473 2084890.179),61.594926,22,8.391288e+06,3.328678e+06
1481132,382.805856,16,google,0.9185,"POLYGON ((8387356.037 3298266.396, 8387356.557...",75.344910,28.390703,75.34491042205822:28.390703116146657,6012.615957,7.102821,...,39,53,382.805856,0.063667,0.012899,POINT (860456.01 2057997.339),52.110714,17,8.387357e+06,3.298322e+06


In [29]:
for category, road_types in roads_categories.items():

    print(f'Processing road_types: {road_types}')
    
    filtered_roads_df = final_gdf[final_gdf['highway'].isin(road_types)]
    print(f'Unexploded road geometries amount: {len(filtered_roads_df)}')
    
    filtered_roads_df = explode_road_geometry(filtered_roads_df)
    
    print(f'Exploded road geometries amount: {len(filtered_roads_df)}')
    road_centroids = filtered_roads_df['geometry_centroid']  

    road_coords = np.array([(point.x, point.y) for point in road_centroids if not point.is_empty])
    
    if len(road_coords) == 0:
        raise ValueError("No valid road centroids found. Check road geometries!")

    road_tree = cKDTree(road_coords)

    house_coords = np.array(list(zip(df_K.centroid_x, df_K.centroid_y)))  

    distances, indices = road_tree.query(house_coords, k=1)
    
    distance_col_name = f'distance_to_{category}'
    road_type_col_name = f'nearest_road_type_{category}'
    
    df_K[road_type_col_name] = ''
    for building_idx, (distance, idx) in tqdm(enumerate(zip(distances, indices)), desc='Assigning roads & distances', total=len(distances)):
        
        df_K.loc[building_idx, distance_col_name] = float(distance)
        df_K.loc[building_idx, road_type_col_name] = filtered_roads_df.iloc[idx].highway

Processing road_types: ['motorway', 'trunk_link', 'motorway_link', 'trunk', 'primary', 'primary_link']
Unexploded road geometries amount: 262
Exploded road geometries amount: 14522


Assigning roads & distances: 100%|██████████| 1481134/1481134 [09:15<00:00, 2666.43it/s]


Processing road_types: ['secondary', 'secondary_link']
Unexploded road geometries amount: 45
Exploded road geometries amount: 3367


Assigning roads & distances: 100%|██████████| 1481134/1481134 [09:13<00:00, 2676.40it/s]


Processing road_types: ['tertiary', 'tertiary_link']
Unexploded road geometries amount: 220
Exploded road geometries amount: 16374


Assigning roads & distances: 100%|██████████| 1481134/1481134 [09:15<00:00, 2666.87it/s]


Processing road_types: ['residential', 'footway', 'service', 'unclassified', 'living_street', 'steps', 'path', 'track', 'pedestrian', 'cycleway', 'raceway', 'bridleway', 'construction', 'services', 'bus_stop', 'road', 'rest_area', 'yes', 'emergency_access_point', 'corridor', 'junction', 'proposed', 'minor']
Unexploded road geometries amount: 6840
Exploded road geometries amount: 141209


Assigning roads & distances: 100%|██████████| 1481134/1481134 [09:16<00:00, 2662.36it/s]


CHECKS

In [30]:
distance_columns = ['distance_to_1', 'distance_to_2', 'distance_to_3', 'distance_to_4']

df_K[distance_columns].describe()

,distance_to_1,distance_to_2,distance_to_3,distance_to_4
count,1.481134e+06,1.481134e+06,1.481134e+06,1.481134e+06
mean,4.607060e+03,1.953271e+04,6.339567e+03,6.311113e+02
std,5.042892e+03,1.437986e+04,7.451161e+03,9.901910e+02
min,1.039019e+00,8.199487e-01,3.769349e-01,2.175044e-02
25%,6.452146e+02,8.588765e+03,5.242949e+02,5.627048e+01
50%,3.028554e+03,1.742142e+04,3.832133e+03,1.929500e+02
75%,6.827231e+03,2.822373e+04,9.536064e+03,7.153352e+02
max,3.037952e+04,7.748557e+04,4.322830e+04,8.758021e+03


In [31]:
for cat, limit in category_bbox_size.items():
    
    df_K[f'distance_to_{cat}'] = df_K[f'distance_to_{cat}'].clip(lower=0, upper=limit)

In [32]:
distance_columns = ['distance_to_1', 'distance_to_2', 'distance_to_3', 'distance_to_4']

df_K[distance_columns].describe()

,distance_to_1,distance_to_2,distance_to_3,distance_to_4
count,1.481134e+06,1.481134e+06,1.481134e+06,1.481134e+06
mean,2.825958e+03,3.609418e+03,1.984561e+03,5.190414e+02
std,2.020138e+03,1.047013e+03,1.231393e+03,6.566903e+02
min,1.039019e+00,8.199487e-01,3.769349e-01,2.175044e-02
25%,6.452146e+02,4.000000e+03,5.242949e+02,5.627048e+01
50%,3.028554e+03,4.000000e+03,3.000000e+03,1.929500e+02
75%,5.000000e+03,4.000000e+03,3.000000e+03,7.153352e+02
max,5.000000e+03,4.000000e+03,3.000000e+03,2.000000e+03


In [33]:
df_K.columns

Index(['perimeter_in_meters', 'building_faces', 'bf_source', 'confidence',
       'geometry', 'longitude', 'latitude', 'id', 'area_in_meters',
       'height_mean', 'height_median', 'height_max', 'height', 'floors',
       'gfa_in_meters', 'urban_split', 'ghsl_smod', 'elevation',
       'building_density_50', 'building_density_100', 'building_density_250',
       'building_density_500', 'building_perimeter_in_meters_new',
       'perimeter_to_area_ratio', 'normalized_perimeter_to_area_ratio',
       'centroid', 'radius_m', 'num_vertices', 'centroid_x', 'centroid_y',
       'nearest_road_type_1', 'distance_to_1', 'nearest_road_type_2',
       'distance_to_2', 'nearest_road_type_3', 'distance_to_3',
       'nearest_road_type_4', 'distance_to_4'],
      dtype='object')

In [34]:
fixed_radius_by_category = {
    1: 500,
    2: 400,
    3: 300,
    4: 200,
}

### Density of roads

In [35]:
def explode_road_geometry_without_index(df):
    
    road_rows = []
    for row_idx, row in df.to_dict(orient='index').items():
        
        for x, y in row['geometry'].coords:
            current_row = row.copy()
            current_row['geometry_centroid'] = shapely.Point(x, y)
            current_row['row_idx'] = row_idx
            road_rows.append(current_row)

    result_df = pd.DataFrame.from_dict(road_rows)  
    
    result_df['centroid_x'] = result_df.geometry.apply(lambda g: g.centroid.xy[0][0])
    result_df['centroid_y'] = result_df.geometry.apply(lambda g: g.centroid.xy[1][0]) 
    
    return result_df

In [36]:
filtered_roads = explode_road_geometry_without_index(final_gdf)

In [37]:
road_centroids = filtered_roads['geometry_centroid'] 
road_coords = np.array([(point.x, point.y) for point in road_centroids if not point.is_empty])
   
road_tree = cKDTree(road_coords)

In [38]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
fixed_radius_by_category = {
    1: 500,
    2: 400,
    3: 300,
    4: 200,
    5: 100,
}

In [40]:
roads_categories_ = {
    4: ['residential', 'footway', 'service', 'unclassified','living_street','steps','path','track','pedestrian','cycleway','raceway','bridleway','construction','services','bus_stop','road','rest_area','yes','emergency_access_point','corridor','junction','proposed','minor'],
    5: ['residential', 'footway', 'service', 'unclassified','living_street','steps','path','track','pedestrian','cycleway','raceway','bridleway','construction','services','bus_stop','road','rest_area','yes','emergency_access_point','corridor','junction','proposed','minor']
    }

In [41]:
def compute_road_density(road_tree, building_x, building_y, radius, category_roads_df):
    # Get nearby roads
    nearby_indices = road_tree.query_ball_point((building_x, building_y), radius)
    
    building_radius_polygon = shapely.Point(building_x, building_y).buffer(radius)
    
    if not nearby_indices:
        return 0  

    # Get total road length within radius
    
    road_idxs = list(set(filtered_roads.loc[nearby_indices].row_idx))
    
    filtered_roads_df = category_roads_df[category_roads_df.road_index.isin(road_idxs)].copy()
    
    filtered_roads_df['geometry'] = filtered_roads_df['geometry'].apply(lambda g: g.intersection(building_radius_polygon))
    total_road_length = filtered_roads_df.geometry.length.sum()
    
    # Compute buffer area
    buffer_area = np.pi * (radius ** 2)  # Circle area formula πr²
    
    # Compute density: road length per km²
    return (total_road_length / buffer_area) * 1e6  # Convert to km/km²

building_coords = np.array(list(zip(df_K.centroid_x, df_K.centroid_y)))

for category, road_types in roads_categories_.items():

    fixed_radius = fixed_radius_by_category[category]
    
    category_roads_df = final_gdf[final_gdf.highway.isin(road_types)]
    # df_K[f"road_density_for_{category}_fixed"] = df_K[['centroid_x', 'centroid_y']].apply(lambda row: compute_road_density(row.centroid_x, row.centroid_y, fixed_radius), axis=1)
    # [ for x, y in tqdm(building_coords, total=len(building_coords))]
#  Compute road density for all buildings (Vectorized, fast)
    df_K[f"road_density_for_{category}_fixed"] = [compute_road_density(road_tree, x, y, fixed_radius, category_roads_df) for x, y in tqdm(building_coords, total=len(building_coords), desc=f'Counting for fixed radius: {fixed_radius}')]

Counting for fixed radius: 100: 100%|██████████| 1481134/1481134 [20:27<00:00, 1207.09it/s]


In [42]:
density_columns = ['road_density_for_4_fixed', 'road_density_for_5_fixed']

df_K[density_columns].describe()

,road_density_for_4_fixed,road_density_for_5_fixed
count,1.481134e+06,1.481134e+06
mean,3.389295e+03,3.610111e+03
std,5.396399e+03,6.342813e+03
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,3.198328e+02,0.000000e+00
75%,4.536537e+03,5.842621e+03
max,3.475631e+04,4.462932e+04


SQN + NUMBER OF FACES

In [43]:

df_K["SQN"] = (4 * np.sqrt(df_K["area_in_meters"]) / df_K["building_perimeter_in_meters_new"])
df_K.columns

# #no of faces
df_K["faces"] = df_K['num_vertices'] - 1
df_K.loc[df_K["faces"] > 20, "faces"] = 20


SAVE

In [45]:
df_K.to_parquet("Churu_height_NEW_GHSL_SMOD_elevations_BDensity_radius.parquet")

In [46]:
df_K.head()

,perimeter_in_meters,building_faces,bf_source,confidence,geometry,longitude,latitude,id,area_in_meters,height_mean,...,nearest_road_type_2,distance_to_2,nearest_road_type_3,distance_to_3,nearest_road_type_4,distance_to_4,road_density_for_4_fixed,road_density_for_5_fixed,SQN,faces
0,7.243843,4,google,0.7724,"POLYGON ((8263508.924 3214523.343, 8263508.858...",74.232350,27.726409,74.23234975129046:27.72640873664523,2.504874,0.000000,...,secondary,4000.000000,tertiary,74.101930,unclassified,29.285147,2438.881053,5049.211582,0.873945,4
1,9.011801,4,google,0.7108,"POLYGON ((8331698.419 3359491.171, 8331698.076...",74.844915,28.873013,74.84491545220574:28.873013019801967,2.513892,1.000000,...,secondary,347.719747,tertiary,665.791676,residential,22.933933,8904.525050,15475.334500,0.703755,4
2,6.354292,4,google,0.7123,"POLYGON ((8262863.432 3202487.532, 8262863.149...",74.226555,27.630664,74.22655544425939:27.630663527650903,2.514733,1.722222,...,secondary,4000.000000,tertiary,3000.000000,track,1176.828824,0.000000,0.000000,0.998249,4
3,6.957619,4,google,0.7604,"POLYGON ((8350523.934 3291257.556, 8350523.917...",75.014020,28.334861,75.0140202674914:28.334861398658834,2.529426,0.000000,...,secondary,2065.096821,tertiary,1574.174846,unclassified,154.697820,3091.282198,0.000000,0.914346,4
4,8.217702,4,google,0.7845,"POLYGON ((8301105.491 3221600.715, 8301105.038...",74.570094,27.782683,74.57009354210085:27.782682537523247,2.530687,2.500000,...,secondary,4000.000000,tertiary,79.032166,unclassified,118.088283,4247.384886,0.000000,0.774335,4
